In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cPickle as pickle
import sqlite3

In [3]:
Veri_Seti3_201701_In = pd.read_csv('./Dataset 3/Dataset 3_201701_In.csv')

In [4]:
pd.DataFrame(Veri_Seti3_201701_In, index = [0,1,2,3,4])

,CALLER_ID,TIMESTAMP,ID,CITY_ID
0,1100144895,02-01-2017 12:52,568,7
1,1100144895,02-01-2017 20:40,568,7
2,1100144895,02-01-2017 21:41,568,7
3,1100144895,03-01-2017 14:57,828,7
4,1100144895,05-01-2017 10:01,568,7


## Extract information from January dataset

In [5]:
JanuaryIn = np.asarray(Veri_Seti3_201701_In)

In [6]:
length, width = JanuaryIn.shape

In [7]:
print ('there are {} calls recorded in this dataset'.format(length))

there are 998625 calls recorded in this dataset


### Extract timestamp

In [8]:
timestamp = pd.to_datetime(JanuaryIn[0:length,1], format="%d-%m-%Y %H:%M")

In [9]:
date = np.asarray(timestamp.date)

### Extract called ID

In [10]:
callerID = JanuaryIn[:,0]

In [11]:
callerID = callerID.astype(str)

### Extract prefix

In [12]:
prefix = []
for i in callerID:
    prefix.append(i[0])

prefix = np.asarray(prefix)
prefix = prefix.astype(int)

In [13]:
unique, counts = np.unique(prefix, return_counts=True)
print ('there are {} refugee calls in this dataset'.format(counts[1]))

there are 758728 refugee calls in this dataset


### Extract city ID

In [14]:
cityID = JanuaryIn[:,3]

## Analyse data

In [15]:
to_analyse = np.unique(callerID[prefix==2])

Accessing date

In [16]:
h1 = callerID[timestamp.hour >= 18]
h2 = callerID[timestamp.hour <= 7]
h = np.append(h1,h2)
overlap = np.intersect1d(h,to_analyse)

In [17]:
evening_location = {}
for i in overlap:
    evening_location[str(i)] = {
        'cities':[],
        'dates':[]
    }

In [18]:
import progressbar

In [19]:
#bottleneck - fix this
bar = progressbar.ProgressBar(maxval=len(overlap), \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()

counter = 0
for i in overlap:
    location =  np.where(callerID == i)
    cities = cityID[location[0]]
    evening_location[i]['cities'] = cities
    dates = date[location[0]]
    evening_location[i]['dates'] = dates
    counter += 1
    #if counter%100 == 0:
    #    print (counter)
    bar.update(counter)
#    sleep(0.1)
bar.finish()

[========================================================================] 100%


Save evening_locations file

In [20]:
with open("evening_location.pkl", "wb") as fp:   #Pickling
    pickle.dump(evening_location, fp, 1)

In [21]:
with open("evening_location.pkl", "rb") as fp:   # Unpickling
    evening_location = pickle.load(fp)

In [22]:
mobile = [] #find the refugees who spend their time in more that one city in the evening over the month
for i in evening_location:
    unique, counts = np.unique(evening_location[i]['cities'], return_counts=True)
    if len(unique) > 1:
        mobile.append(i)

In [23]:
print ('there are {} callers out ot 50,000 refugees who moved cities'.format(len(mobile)))

there are 1651 callers out ot 50,000 refugees who moved cities


Calculated cities moved between (i.e. were in 'residence' for more that 4 nights) 

In [29]:
for i in mobile:
    moved = [evening_location[i]['cities'][0]]
    date_moved = evening_location[i]['dates'][0]
    for j in range(1, len(evening_location[i]['cities'])):
        diff = np.timedelta64(evening_location[i]['dates'][j-1]-date_moved)
        diff = int(diff.astype('timedelta64[D]') / np.timedelta64(1, 'D'))
        if diff >=4 and evening_location[i]['cities'][j-1] != moved[-1]:
            moved.append(evening_location[i]['cities'][j-1])
            date_moved = evening_location[i]['dates'][j]
    evening_location[i]['moved'] = moved

In [33]:
low_mobility = []
medium_mobility = []
high_mobility = []
for i in mobile:
    if len(evening_location[i]['moved']) <=2:
        low_mobility.append(i)
    elif 2 < len(evening_location[i]['moved']) <=5:
        medium_mobility.append(i)
    else:
        high_mobility.append(i)

In [34]:
print ('There are {} refugees classed as low mobility'.format(len(low_mobility)))
print ('There are {} refugees classed as medium mobility'.format(len(medium_mobility)))
print ('There are {} refugees classed as high mobility'.format(len(high_mobility)))

There are 835 refugees classed as low mobility
There are 816 refugees classed as medium mobility
There are 0 refugees classed as high mobility


Analysis for cities from which refugees leave

In [72]:
bad_cities = np.zeros(81)

In [73]:
for i in mobile:
    moved = evening_location[i]['moved']
    unique, counts = np.unique(moved, return_counts=True)
    for j in range(len(unique)):
        # think about criterion for repeated city!!!
        if counts[j] == 1:
            bad_cities[unique[j]-1] += 1

In [74]:
bad_cities

array([146.,  31.,  20.,   2.,   8., 132.,  77.,   4.,  28.,  30.,   8.,
         2.,   5.,  23.,  10., 107.,  14.,  12.,   6.,  17.,  18.,  15.,
        11.,   5.,   6.,  26., 161.,   7.,   2.,   1.,  83.,  22., 115.,
       467., 108.,   6.,  11.,  16.,  15.,   9., 179.,  55.,   9.,   7.,
        47.,  34.,  14.,  21.,   3.,   6.,  12.,   6.,   2.,  48.,   9.,
         3.,   3.,   6.,  59.,   5.,   9.,   2.,  80.,   4.,  11.,   3.,
        10.,  12.,   0.,   8.,  14.,   7.,   6.,   1.,   1.,   1.,  47.,
        11.,  34.,  41.,  19.])

In [75]:
city_mapping = pd.read_csv('./Mapping IDs/Dataset3_City_Mapping.csv')

In [76]:
bad_cities_names = {}

In [77]:
city_mapping = np.asarray(city_mapping)

In [78]:
for i in range(len(city_mapping)):
    bad_cities_names[city_mapping[i,1]] = bad_cities[i]

In [79]:
print ('The worst cities for refugees moving out of are:')
for i in bad_cities_names:
    if bad_cities_names[i] >=50:
        print i

The worst cities for refugees moving out of are:
HATAY
SANLIURFA
GAZIANTEP
MERSIN
KOCAELI
ANTALYA
ANKARA
ADANA
TEKIRDAG
ISTANBUL
BURSA
KONYA
IZMIR


In [40]:
print ([x for x,_ in sorted(zip(evening_location[mobile[0]]['dates'],evening_location[mobile[0]]['cities']))])

[datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 1), datetime.date(2017, 1, 2), datetime.date(2017, 1, 2), datetime.date(2017, 1, 2), datetime.date(2017, 1, 2), datetime.date(2017, 1, 2), datetime.date(2017, 1, 2), datetime.date(2017, 1, 2), datetime.date(2017, 1, 2), datetime.date(2017, 1, 2), datetime.date(2017, 1, 2), datetime.date(2017, 1, 2), datetime.date(2017, 1, 2), datetime.date(2017, 1, 3), 

### Compare against non-refugee population

In [ ]:
non_to_analyse = np.unique(callerID[prefix==1])

In [ ]:
non_h1 = callerID[timestamp.hour >= 18]
non_h2 = callerID[timestamp.hour <= 7]
non_h = np.append(non_h1,non_h2)
non_overlap = np.intersect1d(non_h,non_to_analyse)

In [ ]:
non_evening_location = {}
for i in non_overlap:
    non_evening_location[str(i)] = {
        'cities':[],
        'dates':[]
    }

In [ ]:
##bottleneck - could parallelise better
for i in non_overlap:
    location =  np.where(callerID == i)
    cities = cityID[location[0]]
    non_evening_location[i]['cities'] = cities
    dates = date[location[0]]
    non_evening_location[i]['dates'] = dates

In [ ]:
with open("non_evening_location.txt", "wb") as fp:   #Pickling
    pickle.dump(non_evening_location, fp)

In [ ]:
with open("non_evening_location.txt", "rb") as fp:   # Unpickling
    non_evening_location = pickle.load(fp)

In [ ]:
non_mobile = [] #find the refugees who spend their time in more that one city in the evening over the month
for i in non_evening_location:
    unique, counts = np.unique(non_evening_location[i]['cities'], return_counts=True)
    if len(unique) > 1:
        non_mobile.append(i)

In [ ]:
print ('there are {} callers out ot 50,000 non-refugees who moved cities'.format(len(non_mobile)))

In [ ]:
print ('there are {} times as many refugees who moved cities as non-refugees'.format(
    round(float(len(mobile))/float(len(non_mobile)),2)))